In [82]:
import gdax
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import time
import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)
import pytz
from datetime import datetime

In [4]:
#Auth
public_client=gdax.PublicClient()

In [72]:
#The prices collected every hour
time_data_BTC=public_client.get_product_historic_rates('BTC-USD', start='2018-02-12T16:42:48Z', end='2018-04-20T01:00:00Z', granularity=21600)
time.sleep(5)
time_data_ETH=public_client.get_product_historic_rates('ETH-USD', start='2018-02-12T16:42:48Z', end='2018-04-20T01:00:00Z', granularity=21600)
time.sleep(5)
time_data_BCH=public_client.get_product_historic_rates('BCH-USD', start='2018-02-12T16:42:48Z', end='2018-04-20T01:00:00Z', granularity=21600)
time.sleep(5)
time_data_LTC=public_client.get_product_historic_rates('LTC-USD', start='2018-02-12T16:42:48Z', end='2018-04-20T01:00:00Z', granularity=21600)

In [73]:
#Will Collect the following data
df_btc=pd.DataFrame(time_data_BTC, columns=["Time", "low","high", "open","close", "vol"])
df_eth=pd.DataFrame(time_data_ETH, columns=["Time", "low","high", "open","close", "vol"])
df_bch=pd.DataFrame(time_data_BCH, columns=["Time", "low","high", "open","close", "vol"])
df_ltc=pd.DataFrame(time_data_LTC, columns=["Time", "low","high", "open","close", "vol"])

avg_btc=[]
avg_eth=[]
avg_bch=[]
avg_ltc=[]


#Will take the average of the open and close of the 15 min time frame.
avg_btc=(df_btc['open']+df_btc['close'])/2
avg_eth=(df_eth['open']+df_eth['close'])/2
avg_bch=(df_bch['open']+df_bch['close'])/2
avg_ltc=(df_ltc['open']+df_ltc['close'])/2

#since the dataframe is from the end to start, this will reverse the list

new_avg_btc=avg_btc[::-1]
new_avg_eth=avg_eth[::-1]
new_avg_bch=avg_bch[::-1]
new_avg_ltc=avg_ltc[::-1]

In [74]:
#adds the data to the "pricedf" DataFrame. 
pricedf=pd.DataFrame(new_avg_btc, columns=["Average BTC Price"])
pricedf['Average ETH price']=new_avg_eth
pricedf['Average BCH price']=new_avg_bch
pricedf['Average LTC price']=new_avg_ltc
reverse_time=[]
reverse_time=df_btc['Time'][::-1]
pricedf.set_index(reverse_time, inplace=True)

In [78]:
def df_scatter(df, title, seperate_y_axis=False, y_axis_label='', scale='linear', initial_hide=False):
    '''Generate a scatter plot of the entire dataframe'''
    label_arr = list(df)
    series_arr = list(map(lambda col: df[col], label_arr))
    
    layout = go.Layout(
        title=title,
        legend=dict(orientation="h"),
        xaxis=dict(type='date'),
        yaxis=dict(
            title=y_axis_label,
            showticklabels= not seperate_y_axis,
            type=scale
        )
    )
    
    y_axis_config = dict(
        overlaying='y',
        showticklabels=False,
        type=scale )
    
    visibility = 'visible'
    if initial_hide:
        visibility = 'legendonly'
        
    # Form Trace For Each Series
    trace_arr = []
    for index, series in enumerate(series_arr):
        trace = go.Scatter(
            x=series.index, 
            y=series, 
            name=label_arr[index],
            visible=visibility
        )
        
        # Add seperate axis for the series
        if seperate_y_axis:
            trace['yaxis'] = 'y{}'.format(index + 1)
            layout['yaxis{}'.format(index + 1)] = y_axis_config    
        trace_arr.append(trace)

    fig = go.Figure(data=trace_arr, layout=layout)
    py.iplot(fig)
    
combined_df = pricedf
combined_df.pct_change().corr(method='pearson')
def correlation_heatmap(df, title, absolute_bounds=True):
    '''Plot a correlation heatmap for the entire dataframe'''
    heatmap = go.Heatmap(
        z=df.corr(method='pearson').as_matrix(),
        x=df.columns,
        y=df.columns,
        colorbar=dict(title='Pearson Coefficient'),
    )
    
    layout = go.Layout(title=title)
    
    if absolute_bounds:
        heatmap['zmax'] = 1.0
        heatmap['zmin'] = -1.0
        
    fig = go.Figure(data=[heatmap], layout=layout)
    py.iplot(fig)

In [79]:
df_scatter(pricedf,"Price Distribution over 2/12/18-04/20/18")

In [80]:
correlation_heatmap(combined_df.pct_change(), "Cryptocurrency Correlations 02/12/18 - 04/20/18")